In [1]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import nltk

retrieved data of comments and submissions and put them into textfiles

In [4]:
data2 = json.loads(comr.text)

Week of 6/22/20 - Put comments and submission together in a thread...first attempt failed because making requests in a loop is too much for the computer...makeRequest() solves the problem

In [6]:
payload = {'subreddit':'legaladvice', 'size':50,'fields':['id','selftext', 'num_comments'], 'user_removed':'false','mod_removed':'false', 'before':'6d', 'selftext:not':'[removed]', 'selftext:not': '[deleted]'}
r = requests.get('https://api.pushshift.io/reddit/search/submission/', params = payload)
submissiondata = json.loads(r.text)
submissiondata

{'data': [{'id': 'i3l6ab',
   'num_comments': 0,
   'selftext': "Hi everyone -\n\n&amp;#x200B;\n\nThis is my first post ever, so I'm hoping I do this correctly.\n\n&amp;#x200B;\n\nI recently moved out of townhouse. Within the last thirty days, they never contacted us via email or phone number even though they had it on their records. I emailed them 35 days after we moved out and asked for the down deposit back (600  dollars). They responded that sent a closing statement to a previous address on file regarding this and that we OWE them money. This is in the state of Iowa, btw. \n\n&amp;#x200B;\n\nDo I have any ground since they did not notify me within 30 days and I have no idea where they sent the previous mail to?\n\n&amp;#x200B;\n\nThanks!"},
  {'id': 'i3l5za',
   'num_comments': 2,
   'selftext': 'Sorry for any typos/format/rant. Using phone and I\'m in desperation of help as to what to do.\n\nMy workplace is not following any of the rules and guidance against the Coronavirus. They 

In [7]:
len(submissiondata['data'])

50

In [18]:
import time
#mostly from https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
def makeRequest(uri, payload, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri, payload)
        assert response.status_code == 200
        return json.loads(response.content)
    
    current_tries = 1
    while (current_tries < max_retries):
        try:
            time.sleep(1)
            return fire_away(uri)
        except:
            time.sleep(1)
            current_tries+=1
    
    return fire_away(uri)

In [43]:
def makeThreadList(submissiondatalist):
    
    #given submission, retrieves the comments, puts them all together in a dict/list, returns the dict/list
    #submission is submissiondata['data'][i]
    def makeThreadEntry(submission):
        #thread = {'submission' : submission['selftext'], 'comments' : []}
        thread = [submission['selftext']]
        payload = {'fields': 'body', 'size': submission['num_comments'],'link_id': submission['id'],'author':'!LocationBot','mod_removed':'false'}
        comments = makeRequest('https://api.pushshift.io/reddit/search/comment/', payload)
        for i in range(len(comments['data'])):
            thread.append(comments['data'][i]['body'])
        return thread
    
    threads = []
    for j in range(len(submissiondatalist)):
        thread_j = makeThreadEntry(submissiondatalist[j])
        threads.append(thread_j)
    return threads

In [44]:
%%time
#it takes 20s+ to make threadlist of 10 submissions
#~5 minutes for 100 submissions
#50 minutes for 1000 submissions
threadlist = makeThreadList(submissiondata['data'])

CPU times: user 200 ms, sys: 18.3 ms, total: 218 ms
Wall time: 24.2 s


In [45]:
threadlist[2]

['I lived in an apartment that is not giving my deposit back and saying that I owe them money because I left the lease early, by a month. I had verified with the manager verbally when I was required to leave and did as she told me. My lease was signed in the middle of April and I left at the end of March the following year. \nI do not have my old lease agreement because I did not know to keep everything.\n\nThe only proof that I have is an email from my new apartment manager stating that my lease ends at my old apartment on this certain day.\nAlso, I had a friend who lived in the same apartment building and signed her lease in June and left in May. She did receive her deposit back and didn’t have any trouble with them stating she left a month early.\nI left the apartment clean.\n\nDo I have a case to pursue? And would it be wise to pursue it? We are in NM.',
 'It’s about $400',
 '[removed]',
 'How much is your security deposit?']

figure out how to merge results from different requests

In [7]:
#create new submissiondata2 to merge with submissiondata from before
payload = {'subreddit':'legaladvice', 'fields':['id','selftext', 'num_comments'], 'before':'30d'}
r = requests.get('https://api.pushshift.io/reddit/search/submission/', params = payload)
submissiondata2 = json.loads(r.text)
submissiondata2

{'data': [{'id': 'gqzfdz',
   'num_comments': 4,
   'selftext': "Here's the situation. My brother works in sales in the state of Georgia. In march his base pay was reduced from $15 an hour to $8 an hour. 30% of staff was laid off. Executives reduced took a 20% pay cut. He is being pressured not to file for partial unemployment, and everyone that has done so has been fired. Does he have any legal ground to stand on? Thanks!"},
  {'id': 'gqzf02',
   'num_comments': 2,
   'selftext': 'I’ve had so many problems with this teacher, but today in class she started asking students to vote on another students grade. I live in Florida and I’m wondering if this is legal and if not what are my options???'},
  {'id': 'gqzeow',
   'num_comments': 18,
   'selftext': "Kentucky.\n\nI stopped by a friend's house. I noticed a person I didn't like there, so I excused myself. I said goodbye to the host as well as some guests and went to the door and put on my shoes. Person I didnt like saw me, ran in front 

In [8]:
#assigning to new variable doesn't simply make a copy, any change to extended... is also made to submissiondata
#extendedsubmissiondata['data'] = submissiondata['data']
submissiondata['data'].extend(submissiondata2['data'])
submissiondata

{'data': [{'id': 'hfocgb',
   'num_comments': 0,
   'selftext': 'My friend is in the foster care system. She’s been threw hell and back. She’s been threw groups homes. So she got fostered and now there trying to kick her out. She deals with mental issues and they haven’t provided any help to her either. It’s like the world is against her and it makes me so upset and I just want to get her some help or at least some advice'},
  {'id': 'hfobzx',
   'num_comments': 0,
   'selftext': "This is happening in Texas. My dad works construction and a couple days ago after working with 3 other people he finds out that one of them was infected and all 3 where sent home but the company hasn't told my dad anything about getting checked or being put in quarantine. This is strange because a week ago someone else while on at a company meeting was exposed to Covid and the company immediately sent him to get tested and to stay home. Is this discrimination since my dad is a mexican laborer as opposed to a 

determine what to filter out and how: newlines, extra spaces, quotes from other comments, moderators (comments that start with "Your post may have been removed for the following reason(s):")

In [33]:
import re
#m = re.search('\*Your post may have been removed for the following reason\(s\)\:\*',threadlist[48]['comments'][0])

pattern_quoted = r'&gt;[\w\s’.?/()]*\n\n'
#can delete the first word of the next sentence too, but seems like a small loss
pattern_url = r'https://[\w/#:?._~-]*'

#pattern is probably too specific
#will accept any number of \n, \n\n&amp;#x200B;\n\n, and &amp;
#pattern_spaces = r'((\n)+(&amp;(#x200B;)?)*)+|((\n)*(&amp;(#x200B;)?)+)+'
pattern_spaces = r'(\n|(&amp;(#x200B;)?)|-\t)+'
pattern_stars = r'\*+'
pattern_whitespace = r'(\s)+'
pattern_dashes = r'(-)+'

test_comment = "Embarrassing Bonus questions, if you feel like it:\n\n-\tIf I get randomly drug tested at work (barista), can I get arrested? \n-\tIf I get fired, is that going to end up in some kind of legal record that all future employers can see?"
re.sub(pattern_spaces,' ', test_comment)
#m = re.search(pattern_spaces, test_comment)
#m.group()

'Embarrassing Bonus questions, if you feel like it: If I get randomly drug tested at work (barista), can I get arrested?  If I get fired, is that going to end up in some kind of legal record that all future employers can see?'

In [29]:
import re
def preprocess(threadlist):
    
    def clean(threadlist):
        for i in range(len(threadlist)):
            #delete urls
            pattern_url = r'https://[\w/#:?._~-]*'
            #threadlist[i]['submission'] = re.sub(pattern_url,'', threadlist[i]['submission'])
            for j in range(len(threadlist[i])):
                threadlist[i][j] = re.sub(pattern_url,'', threadlist[i][j])
        
            #replace quoted stuff, &gt;[a-zA-Z0-9?.,:;() ]\n\n with whitespace
            #doesn't always have to end in \n\n, if it doesn't, just delete until setence ends
            pattern_quoted = r'&gt;([\w\s’.?/,()]*\n\n|[\w\s’/,()]*)'
            #threadlist[i]['submission'] = re.sub(pattern_quoted,' ', threadlist[i]['submission'])
            for j in range(len(threadlist[i])):
                threadlist[i][j] = re.sub(pattern_quoted,' ', threadlist[i][j])
            
            #replace \n or \n\n&amp;#x200B;\n\n or \n\n or &amp; or \n-\t with white space
            pattern_spaces = r'(\n|(&amp;(#x200B;)?)|-\t)+'
            #threadlist[i]['submission'] = re.sub(pattern_spaces,' ', threadlist[i]['submission'])
            for j in range(len(threadlist[i])):
                threadlist[i][j] = re.sub(pattern_spaces,' ', threadlist[i][j])
    
            #delete stars
            pattern_stars = r'(\*)+'
            #threadlist[i]['submission'] = re.sub(pattern_stars,'', threadlist[i]['submission'])
            for j in range(len(threadlist[i])):
                threadlist[i][j] = re.sub(pattern_stars,'', threadlist[i][j])
           
            #reduce all multiple whitespaces to 1
            pattern_whitespace = r'(\s)+'
            #threadlist[i]['submission'] = re.sub(pattern_whitespace,' ', threadlist[i]['submission'])
            for j in range(len(threadlist[i])):
                threadlist[i][j] = re.sub(pattern_whitespace,' ', threadlist[i][j])
            
    def deleteIrrelevantComments(threadlist):
        mod_pattern = 'Do not reach out to a moderator personally, and do not reply to this message as a comment.'
        for i in range(len(threadlist)):
            j = 0
            while j < len(threadlist[i]):
                if (threadlist[i][j]=='[removed]' or threadlist[i][j]=='[deleted]' or re.search(mod_pattern,threadlist[i][j]) != None or len(threadlist[i][j])==0):
                    del threadlist[i][j]
                j+=1
    
    deleteIrrelevantComments(threadlist)
    clean(threadlist)

In [30]:
%%time
import copy
copythreadlist = copy.deepcopy(threadlist)
preprocess(threadlist)

CPU times: user 4.13 ms, sys: 120 µs, total: 4.25 ms
Wall time: 6.69 ms


In [31]:
threadlist

[['In July of 2016 I was in a car accident in NH, at the time I was 16. I was under the impression that I had car insurance, however that was not the case. I came from a poor family so I didn’t have money for a lawyer so I agreed to pay an amount to both parties every month . After a few years I couldn’t do it anymore so I stopped, fully aware of the consequences. My license was suspended in the state of NH, if I were to get it back I’d need sr-22 and to pay quite a bit of money. For personal reasons I moved to Illinois and legally changed my first and middle name. I haven’t been able to drive for a while now, I miss the freedom. Is there an easy way I can get it back? Can I just get one from Illinois and no one will know? Can I get in trouble for that? What if I took driving school again and applied for a license after? Any help is greatly appreciated, thank you so much for your help!',
  'Both IL and NH are members of the interstate license compact, so as long as NH has your license 

request ~5000 submissions

In [78]:
#how many posts are made a day? about 1000000 per day, 30000-50000 per hour
r = requests.get('https://api.pushshift.io/reddit/search/submission/?before=90d&after=91d&aggs=created_utc&frequency=hour&size=0')
json.loads(r.text)

{'aggs': {'created_utc': [{'doc_count': 43837, 'key': 1585324800},
   {'doc_count': 55200, 'key': 1585328400},
   {'doc_count': 56027, 'key': 1585332000},
   {'doc_count': 54476, 'key': 1585335600},
   {'doc_count': 54545, 'key': 1585339200},
   {'doc_count': 54462, 'key': 1585342800},
   {'doc_count': 49773, 'key': 1585346400},
   {'doc_count': 49272, 'key': 1585350000},
   {'doc_count': 45993, 'key': 1585353600},
   {'doc_count': 43998, 'key': 1585357200},
   {'doc_count': 40958, 'key': 1585360800},
   {'doc_count': 38828, 'key': 1585364400},
   {'doc_count': 36888, 'key': 1585368000},
   {'doc_count': 33337, 'key': 1585371600},
   {'doc_count': 31432, 'key': 1585375200},
   {'doc_count': 29586, 'key': 1585378800},
   {'doc_count': 27700, 'key': 1585382400},
   {'doc_count': 28612, 'key': 1585386000},
   {'doc_count': 34133, 'key': 1585389600},
   {'doc_count': 33910, 'key': 1585393200},
   {'doc_count': 37758, 'key': 1585396800},
   {'doc_count': 41030, 'key': 1585400400},
   {'doc_

In [93]:
def addSample(submissionlist, size, before, after):
    payload = {'fields': ['id','selftext', 'num_comments'],
             'subreddit':'legaladvice', 
             'size': str(size),
             'mod_removed':'false',
             'before':str(before)+'d',
             'after':str(after)+'d'}
    sample = makeRequest('https://api.pushshift.io/reddit/search/submission/', payload)
    submissionlist['data'].extend(sample['data'])
    return submissionlist

In [39]:
import os
#threadlist[i] = {'submission': "", 'comments':[]}
def makeTextFiles(threadlist, nextStartNum):
    dir_name = '/Users/soumyadugg/reddit_topic_modeling/legal_advice_files'
    
    for i in range(len(threadlist)):
        file = os.path.join(dir_name, 'doc'+ str(nextStartNum+i)+'.txt')
        f = open(file,'w')
        
        #should already be edited
        for j in range(len(threadlist[i])):
            f.write(' ' + threadlist[i][j])
        
        f.close()

In [40]:
makeTextFiles(threadlist,0)

In [ ]:
#take 5 samples each size = 1000 and a couple months separated, pre-Covid
submissiondata = {'data':[]}
submissiondata = addSample(submissiondata,1000,540,541)
submissiondata = addSample(submissiondata,1000,480,481)
submissiondata = addSample(submissiondata,1000,420,421)
submissiondata = addSample(submissiondata,1000,360,361)
submissiondata = addSample(submissiondata,1000,300,301)
threadlist = makeThreadList(submissiondata['data'])
preprocess(threadlist)

My previous way of doing things didn't work because making a big list of jsons takes up too much memory. Next method is to add 100 textfiles at a time to the folder, repeating in a loop